# Example: Static inverse free-boundary equilibrium calculations (in ITER)

---

Here we will generate an equilibrium (find coil currents with the inverse solver) in an ITER-like tokamak. 

The machine description comes from files located [here](https://github.com/ProjectTorreyPines/FUSE.jl).

The equilbirium\profile parameters are **completely made up** - please experiment on your own and change them to more realistic values as you please!

### Import packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

### Create the machine object

In [ ]:
# build machine
from freegsnke import build_machine
tokamak = build_machine.tokamak(
    active_coils_path=f"../machine_configs/ITER/ITER_active_coils.pickle",
    passive_coils_path=f"../machine_configs/ITER/ITER_passive_coils.pickle",
    limiter_path=f"../machine_configs/ITER/ITER_limiter.pickle",
    wall_path=f"../machine_configs/ITER/ITER_wall.pickle",
)

In [ ]:
# plot the machine
fig1, ax1 = plt.subplots(1, 1, figsize=(7, 15), dpi=80)
plt.tight_layout()

tokamak.plot(axis=ax1, show=False)
ax1.plot(tokamak.limiter.R, tokamak.limiter.Z, color='k', linewidth=1.2, linestyle="--")
# ax1.plot(tokamak.wall.R, tokamak.wall.Z, color='k', linewidth=1.2, linestyle="-")

ax1.grid(alpha=0.5)
ax1.set_aspect('equal')
ax1.set_xlim(1.1, 12.4)
ax1.set_ylim(-8.3, 8.3)
ax1.set_xlabel(r'Major radius, $R$ [m]')
ax1.set_ylabel(r'Height, $Z$ [m]')

### Instantiate an equilibrium

In [ ]:
from freegsnke import equilibrium_update

eq = equilibrium_update.Equilibrium(
    tokamak=tokamak,       # provide tokamak object
    Rmin=3.2, Rmax=8.8,   # radial range
    Zmin=-5, Zmax=5,   # vertical range
    nx=129,                # number of grid points in the radial direction (needs to be of the form (2**n + 1) with n being an integer)
    ny=129,                # number of grid points in the vertical direction (needs to be of the form (2**n + 1) with n being an integer)
    # psi=plasma_psi
)

### Instantiate a profile object

In [ ]:
# initialise the profiles
from freegsnke.jtor_update import ConstrainBetapIp
profiles = ConstrainBetapIp(
    eq=eq,        # equilibrium object
    betap=0.15,   # poloidal beta
    Ip=11e6,      # plasma current
    fvac=0.5,     # fvac = rB_{tor}
    alpha_m=2.0,  # profile function parameter
    alpha_n=1.0   # profile function parameter
)

### Load the static nonlinear solver

In [ ]:
from freegsnke import GSstaticsolver
GSStaticSolver = GSstaticsolver.NKGSsolver(eq)    

### Constraints

In [ ]:
Rx = 5.02      # X-point radius
Zx = -3.23      # X-point height

Ro = 6.34      # X-point radius
Zo = 0.66      # X-point height

# set desired null_points locations
# this can include X-point and O-point locations
null_points = [[Rx, Ro], [Zx, Zo]]

# set desired isoflux constraints with format 
# isoflux_set = [isoflux_0, isoflux_1 ... ] 
# with each isoflux_i = [R_coords, Z_coords]
isoflux_set = np.array([[
    [4.25455147,
 4.1881875,
 4.2625,
 4.45683769,
 4.78746942,
 5.31835938,
 5.91875,
 6.44275166,
 6.92285156,
 7.35441013,
 7.73027344,
 8.03046875,
 8.19517497,
 8.05673414,
 7.75308013,
 7.37358451,
 6.94355469,
 6.47773438,
 5.99121094,
 5.49433594,
 Rx,
 4.79042969,
 4.56269531,
 4.36038615], 
[0.0,
 1.13554688,
 2.2565134,
 3.16757813,
 3.80507812,
 4.0499021,
 3.93089003,
 3.665625,
 3.31076604,
 2.86875,
 2.3199601,
 1.62832118,
 0.657421875,
 -0.35859375,
 -1.05585937,
 -1.59375,
 -2.03492727,
 -2.41356403,
 -2.75622016,
 -3.08699278,
 Zx,
 -2.40548044,
 -1.55982142,
 -0.67734375]
    ]])
           

# instantiate the freegsnke constrain object
from freegsnke.inverse import Inverse_optimizer
constrain = Inverse_optimizer(null_points=null_points,
                              isoflux_set=isoflux_set)


# remove from the coils available for control the radial field coil 
# eq.tokamak['VS3'].control = False

### The inverse solve

In [ ]:
GSStaticSolver.inverse_solve(eq=eq, 
                     profiles=profiles, 
                     constrain=constrain, 
                     target_relative_tolerance=1e-4,
                     target_relative_psit_update=1e-3,
                    #  max_solving_iterations=150,
                    #  max_iter_per_update=10,
                    #  max_rel_update_size=0.075,
                    #  damping_factor=.99,
                    #  max_rel_psit=.05,
                     verbose=True, # print output
                     l2_reg=1e-14,
                     )


In [ ]:
fig1, ax1 = plt.subplots(1, 1, figsize=(7, 15), dpi=80)

ax1.grid(zorder=0, alpha=0.75)
ax1.set_aspect('equal')
eq.tokamak.plot(axis=ax1,show=False)                                                          # plots the active coils and passive structures
ax1.fill(tokamak.wall.R, tokamak.wall.Z, color='k', linewidth=1.2, facecolor='w', zorder=0)   # plots the limiter
eq.plot(axis=ax1,show=False)                                                                  # plots the equilibrium
constrain.plot(axis=ax1, show=False)                                                          # plots the contraints
ax1.set_xlim(1.1, 12.4)
ax1.set_ylim(-8.3, 8.3)

In [ ]:
eq.tokamak.getCurrents()

# # save coil currents to file
# import pickle
# with open('simple_diverted_currents_PaxisIp.pk', 'wb') as f:
#     pickle.dump(obj=inverse_current_values, file=f)